In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix
from sklearn.metrics.pairwise import cosine_similarity

 # Data preprocessing

In [2]:
# Load data
behaviors = pd.read_csv('./small_training_data/behaviors.tsv', delimiter='\t', header=None)
news = pd.read_csv('./small_training_data/news.tsv', delimiter='\t', header=None)

# Naming columns
behaviors.columns = ["impression_id", "user_id", "time", "history", "impressions"]
news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]

In [3]:
# Remove NaN values in the 'abstract' column
news = news.dropna(subset=['abstract'])

# list of valid news (with some abstract)
valid_news_ids = set(news['news_id'])

In [4]:
# Extracting clicked news from behaviors, this is a column of lists of the clicked news (tagget with 1) for each impression
behaviors['clicked_news'] = behaviors['impressions'].apply(lambda x: [imp.split('-')[0] for imp in x.split() if imp.split('-')[1] == '1'])

In [5]:
#removing unused columns
behaviors = behaviors[["impression_id", "user_id", "clicked_news"]]

In [6]:
# Flattening the clicked news and associating with user_id, that means we divide the lists into one row for each clicked news
clicked_news = behaviors.explode('clicked_news')[['user_id', 'clicked_news']]

In [7]:
#remove non valid news from interactions
clicked_news = clicked_news[clicked_news['clicked_news'].isin(valid_news_ids)]

In [8]:
# Filtering users with more than 4 news clicked since the distribution is 'ultra-skewed'
clicked_news = clicked_news.groupby('user_id').filter(lambda x: len(x) > 4)

In [9]:
# Rename 'clicked_news' column to 'news_id'
clicked_news = clicked_news.rename(columns={'clicked_news': 'news_id'})

In [10]:
print(f"Number of unique clicked news: {clicked_news['news_id'].nunique()}")
print(f"Number of unique users:        {clicked_news['user_id'].nunique()} \n")

Number of unique clicked news: 6522
Number of unique users:        15418 



In [11]:
news = news.reset_index()

## in the cell below we create mappings from real id's to indexes, and create clicked_news_encoded

In [12]:
# Create categorical types without encoding them yet
clicked_news['user_id_cat'] = clicked_news['user_id'].astype("category")
clicked_news['news_id_cat'] = clicked_news['news_id'].astype("category")

# Creating mappings from original IDs to encoded IDs
id_to_user = dict(enumerate(clicked_news['user_id_cat'].cat.categories))
id_to_news = dict(enumerate(clicked_news['news_id_cat'].cat.categories))

# Convert categories to codes (integer encoding)
clicked_news_encoded = pd.DataFrame(columns=['user', 'item'])
clicked_news_encoded['user'] = clicked_news['user_id_cat'].cat.codes
clicked_news_encoded['item'] = clicked_news['news_id_cat'].cat.codes

# Drop the additional categorical columns if they are not needed
clicked_news = clicked_news.drop(columns=['user_id_cat', 'news_id_cat'])

# Creating reverse mappings from original IDs to encoded IDs
user_to_id = {v: k for k, v in id_to_user.items()}
news_to_id = {v: k for k, v in id_to_news.items()}

In [13]:
# Create a sparse user-item interaction matrix
interaction_matrix = coo_matrix((np.ones(clicked_news_encoded.shape[0]),
                                 (clicked_news_encoded['user'], clicked_news_encoded['item'])))

print(f"users: {interaction_matrix.shape[0]} \nitems: {interaction_matrix.shape[1]}")

users: 15418 
items: 6522


In [14]:
#transform into compressed sparse row
interaction_matrix_csr = interaction_matrix.tocsr()

In [15]:
clicked_news_encoded['rating'] = np.ones(len(clicked_news_encoded))

In [16]:
print(clicked_news_encoded)

         user  item  rating
1       14889   756     1.0
5        1803  1190     1.0
5        1803  2565     1.0
9        7888  4810     1.0
10      14512  5700     1.0
...       ...   ...     ...
156963   6291  1261     1.0
156963   6291  4396     1.0
156963   6291  2904     1.0
156963   6291   915     1.0
156963   6291  1137     1.0

[155443 rows x 3 columns]


In [17]:
user_click_counts = clicked_news['user_id'].value_counts().reset_index()
user_click_counts.columns = ['user_id', 'num_of_clicks']
user_click_counts_sorted = user_click_counts.sort_values(by='num_of_clicks', ascending=False)
print(user_click_counts_sorted)

      user_id  num_of_clicks
0      U53220            125
1      U70550            118
2      U63482            109
3      U20833             95
4      U32322             94
...       ...            ...
13230  U91963              5
13231  U65567              5
13232    U417              5
13233  U63788              5
15417   U5480              5

[15418 rows x 2 columns]


# Content based recommendations

In [18]:
# Vectorizing the 'abstract' column of news
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
news_profiles = vectorizer.fit_transform(news['abstract'])

In [19]:
# Reversing the vocabulary dictionary
reverse_vocab = {v: k for k, v in vectorizer.vocabulary_.items()}

print(reverse_vocab.get(3845))

royals


This is a matrix that holds the similarity scores between all pairs of news items. It's a  square matrix where each row and column corresponds to a news item, and the entry [i, j] gives the similarity between news item i and news item j. the diagonal is full of ones for obvious reasons

In [20]:
# Compute the cosine similarity matrix
news_similarity = cosine_similarity(news_profiles, dense_output=False)

In [21]:
all_id_to_news = news['news_id'].to_dict()
all_news_to_id = {v: k for k, v in all_id_to_news.items()}

In [76]:
def recommend_news(user_id, interaction_matrix, news_similarity, user_to_id = user_to_id, top_n=5):
    """
    Recommend top N news items for a given user.
    
    Parameters:
        user_id (str): The user for whom to make recommendations.
        interaction_matrix (csr_matrix): User-item interaction matrix.
        news_similarity (csr_matrix): Item-item similarity matrix.
        top_n (int): Number of items to recommend.
        
    Returns:
        list: List of recommended item indices.
    """
    user_id = user_to_id[user_id]
    
    # Get the user's past interactions (list of news ids that the user has interacted with)
    user_interactions = interaction_matrix[user_id, :].nonzero()[1]
    
    # Sum the similarities of the interacted items 
    similarity_sum = np.sum(news_similarity[user_interactions, :], axis=0) #shape = (n_interactions, all_news)
    
    # Remove already interacted items
    similarity_sum[0, user_interactions] = 0
    
    # Get top N item indices
    recommended_news_ids = np.argsort(similarity_sum)[0, -top_n:][::-1]

    #get similarity scores of top N items
    similarity_scores = [similarity_sum[0, idx] for idx in recommended_news_ids.tolist()[0][::-1]]

    # Convert indices to original news IDs
    recommended_news = [all_id_to_news[idx] for idx in recommended_news_ids.tolist()[0]]
    
    return recommended_news, similarity_scores

In [167]:
def explain_individual(user_id, recommended_items, similarity_scores):

    explanation = f"{user_id}: Based off of your click history and similar content, we recommend these articles:\n{recommended_items}.\nThese are their respective similarity scores:\n{similarity_scores}"

    return explanation

In [168]:
# Recommend for a user
user_id = 'U53220'
recommended_news_indices, similarities = recommend_news(user_id, interaction_matrix_csr, news_similarity, top_n=10)

print(explain_individual(user_id, recommended_news_indices, similarities))

U53220: Based off of your click history and similar content, we recommend these articles:
['N52294', 'N31748', 'N38488', 'N26767', 'N60470', 'N560', 'N19994', 'N21041', 'N43291', 'N31626'].
These are their respective similarity scores:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


## Test set

In [98]:
# Load data
test_behaviors = pd.read_csv('./small_test_data/behaviors.tsv', delimiter='\t', header=None)
test_news = pd.read_csv('./small_test_data/news.tsv', delimiter='\t', header=None)

# Naming columns
test_behaviors.columns = ["impression_id", "user_id", "time", "history", "impressions"]
test_news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]

In [99]:
# Remove NaN values in the 'abstract' column
test_news = test_news.dropna(subset=['abstract'])

# list of valid news (with some abstract)
test_valid_news_ids = set(test_news['news_id'])

In [100]:
# Extracting clicked news from behaviors, this is a column of lists of the clicked news (tagget with 1) for each impression
test_behaviors['clicked_news'] = test_behaviors['impressions'].apply(lambda x: [imp.split('-')[0] for imp in x.split() if imp.split('-')[1] == '1'])

#removing unused columns
test_behaviors = test_behaviors[["impression_id", "user_id", "clicked_news"]]

# Flattening the clicked news and associating with user_id, that means we divide the lists into one row for each clicked news
test_clicked_news = test_behaviors.explode('clicked_news')[['user_id', 'clicked_news']]

#remove non valid news from interactions
test_clicked_news = test_clicked_news[test_clicked_news['clicked_news'].isin(test_valid_news_ids)]

# Rename 'clicked_news' column to 'news_id'
test_data = test_clicked_news.rename(columns={'clicked_news': 'news_id'})

In [101]:
print(f"Number of unique clicked news: {test_data['news_id'].nunique()}")
print(f"Number of unique users:        {test_data['user_id'].nunique()} \n")

Number of unique clicked news: 2115
Number of unique users:        48139 



### Since it's a content-based recommender system we ran into the cold start problem from users present in the test data but not in the training data, so we just recommend them items from the most popular news 

In [102]:
news_popularity = clicked_news['news_id'].value_counts()

# Calculate the click threshold for the top percentile
threshold = np.percentile(news_popularity, 98)

# Get the most popular news items
popular_news = news_popularity[news_popularity >= threshold].index.tolist()

## Model evaluation - Precision@k and Recall@k and nCDG@k

In [103]:
recommendations = {} #{'user_id': [list of recommended news_ids]}

In [104]:
# Set of user_ids from test_data and training data
test_data_user_ids = set(test_data['user_id'].unique())

train_data_user_ids = set(clicked_news['user_id'].unique())

In [105]:
for user_id in test_data_user_ids:
    if user_id not in train_data_user_ids:
        # New user or user not in test data
        recommendations[user_id] = random.sample(popular_news, 5)  # Recommend random news from popular news items
    else:
        recommended_news_indices = recommend_news(user_id, interaction_matrix_csr, news_similarity, top_n=10)
        recommendations[user_id] = [idx for idx in recommended_news_indices]

In [106]:
def precision_at_k(recommended_list, relevant_list, k):
    """
    Compute Precision at K.
    
    Parameters:
        recommended_list (list): List of recommended items.
        relevant_list (list): List of relevant items.
        k (int): Number of recommendations to consider.
        
    Returns:
        float: Precision at K score.
    """
    return len(set(recommended_list[:k]) & set(relevant_list)) / k

In [107]:
def recall_at_k(recommended_list, relevant_set, k):
    """Return the recall at k."""
    return len(set(recommended_list[:k]) & set(relevant_set)) / len(relevant_set)

In [108]:
def ndcg_at_k(recommended_list, relevant_set, k):
    """Return the NDCG at k."""
    dcg = 0
    idcg = sum([1 / math.log(i + 2, 2) for i in range(min(k, len(relevant_set)))])
    for i, item in enumerate(recommended_list[:k]):
        if item in relevant_set:
            dcg += 1 / math.log(i + 2, 2)
    return dcg / idcg

In [149]:
k = 5 # or any value you choose

precision_values = []
recall_values = []
ndcg_values = []

for user_id, recommended_items in recommendations.items():
    relevant_items = tuple(test_data[test_data['user_id'] == user_id]['news_id'].tolist())

    
    if relevant_items:  # if the user has any relevant items
        
        precision = precision_at_k(recommended_items, relevant_items, k)
        recall = recall_at_k(recommended_items, relevant_items, k)
        #ndcg = ndcg_at_k(recommended_items, relevant_items, k)
        
        precision_values.append(precision)
        recall_values.append(recall)
        #ndcg_values.append(ndcg)

# Averaging across all users to get the final metric value
mean_precision = np.mean(precision_values)
mean_recall = np.mean(recall_values)
#mean_ndcg = np.mean(ndcg_values)

print(f"Mean Precision@{k}: {mean_precision:.4f}")
print(f"Mean Recall@{k}: {mean_recall:.4f}")
#print(f"Mean NDCG@{k}: {mean_ndcg:.4f}")

Mean Precision@5: nan
Mean Recall@5: nan


C:\Users\bcroe\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bcroe\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


I think it's due to the sparsity and cold start problem, we have many users without interactions

In [150]:
filtered_test_data = test_data.groupby('user_id').filter(lambda x: len(x) > 10)

In [151]:
test_data_user_ids

{'U10103',
 'U10271',
 'U1030',
 'U10601',
 'U10605',
 'U10642',
 'U10677',
 'U1098',
 'U11160',
 'U11221',
 'U11469',
 'U11680',
 'U11713',
 'U11885',
 'U12170',
 'U1226',
 'U12835',
 'U13024',
 'U13314',
 'U13717',
 'U14088',
 'U14271',
 'U15005',
 'U15187',
 'U15377',
 'U15384',
 'U15699',
 'U16167',
 'U16217',
 'U16218',
 'U16283',
 'U1634',
 'U16748',
 'U16825',
 'U16888',
 'U17206',
 'U17318',
 'U17516',
 'U17797',
 'U17970',
 'U18277',
 'U18465',
 'U18498',
 'U18677',
 'U18785',
 'U1886',
 'U18874',
 'U18895',
 'U19378',
 'U19593',
 'U19653',
 'U19774',
 'U19858',
 'U19913',
 'U1994',
 'U2006',
 'U20605',
 'U20689',
 'U21026',
 'U21106',
 'U21533',
 'U21544',
 'U21921',
 'U22324',
 'U22530',
 'U22637',
 'U23121',
 'U23206',
 'U23276',
 'U23446',
 'U23541',
 'U23568',
 'U23633',
 'U23738',
 'U23857',
 'U23931',
 'U23966',
 'U24000',
 'U24172',
 'U24311',
 'U25152',
 'U25358',
 'U25458',
 'U25472',
 'U25541',
 'U25918',
 'U26004',
 'U26286',
 'U26364',
 'U2638',
 'U2665',
 'U26755

In [158]:
recommendations = {} #{'user_id': [list of recommended news_ids]}

# Set of user_ids from test_data and training data
test_data_user_ids = set(filtered_test_data['user_id'].unique())

train_data_user_ids = set(clicked_news['user_id'].unique())

for user_id in test_data_user_ids:
    if user_id not in train_data_user_ids:
        # New user or user not in test data
        recommendations[user_id] = random.sample(popular_news, 5)  # Recommend random news from popular news items
    else:
        recommended_news_indices = recommend_news(user_id, interaction_matrix_csr, news_similarity, top_n=10)
        recommendations[user_id] = [idx for idx in recommended_news_indices]

KeyError: 'user_id'

In [159]:
k = 5  # or any value you choose

precision_values = []
recall_values = []
ndcg_values = []

for user_id, recommended_items in recommendations.items():
    relevant_items = test_data[test_data['user_id'] == user_id]['news_id'].tolist()
    
    if relevant_items:  # if the user has any relevant items
        
        precision = precision_at_k(recommended_items, relevant_items, k)
        recall = recall_at_k(recommended_items, relevant_items, k)
        #ndcg = ndcg_at_k(recommended_items, relevant_items, k)
        
        precision_values.append(precision)
        recall_values.append(recall)
        #ndcg_values.append(ndcg)

# Averaging across all users to get the final metric value
mean_precision = np.mean(precision_values)
mean_recall = np.mean(recall_values)
#mean_ndcg = np.mean(ndcg_values)

print(f"Mean Precision@{k}: {mean_precision:.4f}")
print(f"Mean Recall@{k}: {mean_recall:.4f}")
#print(f"Mean NDCG@{k}: {mean_ndcg:.4f}")

Mean Precision@5: nan
Mean Recall@5: nan


C:\Users\bcroe\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bcroe\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### In fact we can see that is precision increases if we consider users with more than n = 10 interactions

# Group recommendations

In [160]:
from lenskit.algorithms.als import ImplicitMF

In [161]:
# Load JSON string from a file
with open("grouped_dict.json", "r") as f:
    grouped_dict_json = f.read()

# Convert the JSON string back to a dictionary
groups_dict = json.loads(grouped_dict_json)

In [162]:
all_clicks = clicked_news_encoded.reset_index()
all_clicks = all_clicks[['user', 'item', 'rating']]

In [163]:
# Create an implicit feedback model with 50 features and 20 iterations
model = ImplicitMF(features=50, iterations=20, reg=0.1)

# Train the model
model.fit(all_clicks)

In [164]:
def group_recommendations(user_group, model, all_items, user_item_data):
    #getting user_idexes
    user_group = [user_to_id[user] for user in user_group if user in user_to_id]
    
    # A dictionary to aggregate scores
    aggregated_scores = {}

    # A set to collect items already read by any user in the group
    read_by_group = set()

    for user in user_group:
        # Get individual user recommendations
        recs_user = model.predict_for_user(user, all_items)
        
        # Exclude items already read by the user
        read_items = user_item_data[user_item_data['user'] == user]['item'].tolist()
        recs_user = recs_user.drop(read_items, errors='ignore')

        # Add these read items to the group set
        read_by_group.update(read_items)

        # Aggregate scores
        for item, score in recs_user.items():
            if item in aggregated_scores:
                aggregated_scores[item] += score
            else:
                aggregated_scores[item] = score

    # Convert to pandas series for easier manipulation
    aggregated_scores = pd.Series(aggregated_scores)

    # Remove items read by any user in the group
    aggregated_scores = aggregated_scores.drop(list(read_by_group), errors='ignore')

    # Sort and get the top items
    top_items = aggregated_scores.sort_values(ascending=False).head(10)

    # Convert the index to news ids using the dictionary
    top_items.index = top_items.index.map(id_to_news)
    
    return top_items

In [119]:
def explain_group_recs(group_users, news, top_items):
    explanations = {}
    
    for item in top_items:
        # Check for topic
        filtered_news = news.loc[news['news_id'] == item, 'subcategory']
        if not filtered_news.empty:
            topic = filtered_news.values[0]
        else:
            topic = "Unknown"  
        
        user_group = [user for user in group_users if user in id_to_user]
        
        # Generate explanation string
        explanations[item] = f"Recommended because users {group_users} showed a high average preference score for {topic}."

    return explanations

In [120]:
all_items = clicked_news_encoded.item.unique().tolist()

group = groups_dict['Group 6']
group

['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596']

In [121]:
top_10_group_recs = group_recommendations(group, model, all_items, all_clicks)

print("Top 10 recommended items for the group:")
print(top_10_group_recs)

Top 10 recommended items for the group:
N7821     4.451115
N49685    4.123510
N39317    3.916860
N64916    3.542837
N59252    3.521523
N33885    3.288971
N36261    3.244612
N12029    3.075880
N35305    3.009481
N62366    2.955866
dtype: float64


In [122]:
explanations = explain_group_recs(group, news, top_10_group_recs.index)

In [123]:
explanations

{'N7821': "Recommended because users ['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596'] showed a high average preference score for football_nfl.",
 'N49685': "Recommended because users ['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596'] showed a high average preference score for music-celebrity.",
 'N39317': "Recommended because users ['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596'] showed a high average preference score for newspolitics.",
 'N64916': "Recommended because users ['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596'] showed a high average preference score for newscrime.",
 'N59252': "Recommended because users ['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596'] showed a high average preference score for newspolitics.",
 'N33885': "Recommended because users ['U69084', 'U31631', 'U57214', 'U21331', 'U1331', 'U64554', 'U42643', 'U80596'

# Survey recommendations

In [124]:
survey_news = ["N55468", "N8091", "N61914", "N7857", "N32907", "N33584", "N50566", "N47845", "N20212", "N37038", "N39173", "N54950", "N63975", "N28867", "N62996",
               "N35703", "N9680", "N50299", "N61837", "N59295", "N13113", "N57018", "N48320", "N49981", "N11200", "N10470", "N46481", "N27435", "N39125", "N24808",
               "N1587", "N22605", "N10886", "N52386", "N52620", "N30867", "N47140", "N29552", "N55528", "N55610"]

In [125]:
#news for which we don't have the mapping for
invalid_list = [item for item in survey_news if item not in all_news_to_id]

In [126]:
print(invalid_list)

['N63975', 'N28867', 'N57018', 'N46481', 'N22605']


In [128]:
survey_news_ids = [all_news_to_id[item] for item in survey_news if item in all_news_to_id]

In [129]:
# Create a list of letters A to O
users = [chr(i) for i in range(ord('A'), ord('O')+1)]

# Create a DataFrame with 'user' column filled with letters A to O and 'clicked_news' with empty lists
survey_interactions = pd.DataFrame({
    'user': users,
    'item': [[] for _ in users]  # Create an empty list for each user
})


In [130]:
survey_interactions.at[0, 'item'].extend(["N7857", "N33584", "N50566", "N47845", "N39173", "N54950", "N11200", "N22605", "N52620"])
survey_interactions.at[1, 'item'].extend(["N55468", "N61914", "N50566", "N20212", "N37038", "N39173", "N54950", "N35703", "N50299", "N61837", "N11200", "N10470", "N46481", "N27435", "N1587", "N22605", "N10886", "N52386", "N30867", "N55528"])
survey_interactions.at[2, 'item'].extend(["N55468", "N8091", "N7857", "N32907", "N33584", "N50566", "N47845", "N20212", "N37038", "N39173", "N54950", "N22605", "N10886"])
survey_interactions.at[3, 'item'].extend(["N8091","N7857","N50566","N47845","N20212","N37038","N39173","N54950","N28867","N62996","N35703","N9680","N50299","N61837","N59295","N48320","N46481","N39125","N24808","N22605","N10886","N52386","N52620","N29552","N55528","N55610"])
survey_interactions.at[4, 'item'].extend(["N61914","N47845","N20212","N39173","N28867","N62996","N35703","N61837","N13113","N48320","N11200","N46481","N39125","N24808","N10886","N52386","N47140","N29552","N55528"])
survey_interactions.at[5, 'item'].extend(["N55468","N8091","N7857","N47845","N20212","N35703","N9680","N59295","N48320","N49981","N10470","N24808","N1587","N22605","N10886","N30867"])
survey_interactions.at[6, 'item'].extend(["N55468","N7857","N47845","N62996","N9680","N11200","N10886","N47140","N29552"])
survey_interactions.at[7, 'item'].extend(["N55468","N8091","N7857","N33584","N50566","N47845","N39173","N59295","N11200","N39125","N22605","N10886","N30867","N29552"])
survey_interactions.at[8, 'item'].extend(["N55468","N61914","N50566","N47845","N20212","N37038","N39173","N54950","N62996","N35703","N61837","N59295","N57018","N39125","N24808","N1587","N52620","N29552"])
survey_interactions.at[9, 'item'].extend(["N8091","N7857","N33584","N61837","N57018","N10470","N1587","N10886","N52620","N55528","N55610"])
survey_interactions.at[10, 'item'].extend(["N8091","N61914","N7857","N33584","N47845","N37038","N24808","N10886","N52620","N30867"])
survey_interactions.at[11, 'item'].extend(["N61914","N33584","N20212","N37038","N28867","N57018","N49981","N11200","N10470","N27435","N39125","N24808","N10886","N52386","N52620","N30867","N47140","N29552"])
survey_interactions.at[12, 'item'].extend(["N7857","N32907","N50566","N47845","N62996","N35703","N39125","N24808"])
survey_interactions.at[13, 'item'].extend(["N55468","N8091","N50566","N20212","N62996","N35703","N59295","N57018","N10470","N39125","N22605","N10886","N52620","N30867"])
survey_interactions.at[14, 'item'].extend(["N39173","N54950","N59295","N13113","N10886"])

In [131]:
survey_interactions['item'] = survey_interactions['item'].apply(lambda x: [item for item in x if item not in invalid_list])

In [132]:
survey_interactions

,user,item
0,A,"[N7857, N33584, N50566, N47845, N39173, N54950..."
1,B,"[N55468, N61914, N50566, N20212, N37038, N3917..."
2,C,"[N55468, N8091, N7857, N32907, N33584, N50566,..."
3,D,"[N8091, N7857, N50566, N47845, N20212, N37038,..."
4,E,"[N61914, N47845, N20212, N39173, N62996, N3570..."
5,F,"[N55468, N8091, N7857, N47845, N20212, N35703,..."
6,G,"[N55468, N7857, N47845, N62996, N9680, N11200,..."
7,H,"[N55468, N8091, N7857, N33584, N50566, N47845,..."
8,I,"[N55468, N61914, N50566, N47845, N20212, N3703..."
9,J,"[N8091, N7857, N33584, N61837, N10470, N1587, ..."


In [133]:
# Flattening the clicked news and associating with user_id, that means we divide the lists into one row for each clicked news
survey_interactions = survey_interactions.explode('item')[['user', 'item']]

In [134]:
survey_interactions

,user,item
0,A,N7857
0,A,N33584
0,A,N50566
0,A,N47845
0,A,N39173
...,...,...
14,O,N39173
14,O,N54950
14,O,N59295
14,O,N13113


In [135]:
print(f"Number of unique users:        {survey_interactions['user'].nunique()} \n")
print(f"Number of unique clicked news: {survey_interactions['item'].nunique()}")

Number of unique users:        15 

Number of unique clicked news: 35


In [136]:
#concat new user data
clicked_news.columns = ['user', 'item']
survey_interactions = pd.concat([clicked_news, survey_interactions], ignore_index=True, axis = 0)
# Create categorical types without encoding them yet
survey_interactions['user_id_cat'] = survey_interactions['user'].astype("category")
survey_interactions['news_id_cat'] = survey_interactions['item'].astype("category")

# Creating mappings from original IDs to encoded IDs
id_to_user = dict(enumerate(survey_interactions['user_id_cat'].cat.categories))
id_to_news = dict(enumerate(survey_interactions['news_id_cat'].cat.categories))

# Convert categories to codes (integer encoding)
survey_interactions_encoded = pd.DataFrame(columns=['user', 'item'])
survey_interactions_encoded['user'] = survey_interactions['user_id_cat'].cat.codes
survey_interactions_encoded['item'] = survey_interactions['news_id_cat'].cat.codes

# Drop the additional categorical columns if they are not needed
survey_interactions = survey_interactions.drop(columns=['user_id_cat', 'news_id_cat'])

# Creating reverse mappings from original IDs to encoded IDs
user_to_id = {v: k for k, v in id_to_user.items()}
news_to_id = {v: k for k, v in id_to_news.items()}

# Create a sparse user-item interaction matrix
interaction_matrix = coo_matrix((np.ones(survey_interactions_encoded.shape[0]),
                                 (survey_interactions_encoded['user'], survey_interactions_encoded['item'])))

print(f"users: {interaction_matrix.shape[0]} \nitems: {interaction_matrix.shape[1]}")

#transform into compressed sparse row
interaction_matrix_csr = interaction_matrix.tocsr()

survey_interactions_encoded['rating'] = np.ones(len(survey_interactions_encoded))

print(survey_interactions_encoded)

users: 15433 
items: 6556
         user  item  rating
0       14904   760     1.0
1        1818  1195     1.0
2        1818  2576     1.0
3        7903  4833     1.0
4       14527  5728     1.0
...       ...   ...     ...
155631     14  3230     1.0
155632     14  4978     1.0
155633     14  5454     1.0
155634     14   355     1.0
155635     14    98     1.0

[155636 rows x 3 columns]


In [137]:
user_id = 'B'
recommended_news_indices = recommend_news('A', interaction_matrix_csr, news_similarity, user_to_id, top_n=10)

In [138]:
users = list('ABCDEFGHIJKLMNO')

In [170]:
recommendations = {}


# Iterate through all users and print individual explanations
for user in users:
    recommended_news_indices, similarity_scores = recommend_news(user, interaction_matrix_csr, news_similarity, user_to_id, top_n=5)
    print(explain_individual(user, recommended_news_indices, similarity_scores))
    recommendations[user] = recommended_news_indices

A: Based off of your click history and similar content, we recommend these articles:
['N51923', 'N35778', 'N62171', 'N63004', 'N13589'].
These are their respective similarity scores:
[0.5798973804199299, 0.5538830162498715, 0.525525562409543, 0.5217986832414446, 0.5129110803151521]
B: Based off of your click history and similar content, we recommend these articles:
['N10617', 'N732', 'N64778', 'N723', 'N20167'].
These are their respective similarity scores:
[0.9901153258164977, 0.8059423130067741, 0.7685992367111084, 0.7581549423846585, 0.7543851863944394]
C: Based off of your click history and similar content, we recommend these articles:
['N14598', 'N55628', 'N37062', 'N9470', 'N58055'].
These are their respective similarity scores:
[0.7922353633086818, 0.7434540610570919, 0.7038559853711308, 0.6949625903500387, 0.6857294433946879]
D: Based off of your click history and similar content, we recommend these articles:
['N57715', 'N37062', 'N21212', 'N40099', 'N22556'].
These are their r

In [140]:
recommendations_df = pd.DataFrame.from_dict(recommendations, orient='index')
recommendations_df

,0,1
A,"[N51923, N35778, N62171, N63004, N13589]","[0.5798973804199299, 0.5538830162498715, 0.525..."
B,"[N10617, N732, N64778, N723, N20167]","[0.9901153258164977, 0.8059423130067741, 0.768..."
C,"[N14598, N55628, N37062, N9470, N58055]","[0.7922353633086818, 0.7434540610570919, 0.703..."
D,"[N57715, N37062, N21212, N40099, N22556]","[1.084696778710408, 1.013835934679487, 0.97263..."
E,"[N39961, N36954, N57715, N53891, N22556]","[0.994948332944306, 0.9461104884615255, 0.9322..."
F,"[N9800, N9261, N20536, N57796, N37062]","[0.8740958261609132, 0.7449040884935969, 0.744..."
G,"[N63406, N48316, N40683, N13589, N9470]","[0.5924841002370524, 0.5798973804199299, 0.571..."
H,"[N26457, N22556, N55628, N39961, N57715]","[1.018700477826489, 0.9772576252285645, 0.8616..."
I,"[N3081, N39961, N57715, N22556, N40099]","[0.9973721456321968, 0.9871227724839782, 0.903..."
J,"[N12352, N4244, N732, N56687, N20306]","[0.6567977168684938, 0.6567977168684938, 0.637..."


In [189]:
filtered_news = pd.DataFrame(columns=news.columns)
for recommended_indices_arr in recommendations_df[0]:
    for recommended_index in recommended_indices_arr:
        recommended_item_row = news.loc[news['news_id'] == recommended_index]
        filtered_news = pd.concat([filtered_news, recommended_item_row])
filtered_news

,index,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities
6659,6946,N51923,foodanddrink,newstrends,Founders Tells CBS Beer Release Ticket Holders...,Taproom staff are organizing a peaceful protes...,https://assets.msn.com/labs/mind/AAJl06q.html,"[{""Label"": ""Founders Brewing Company"", ""Type"":...","[{""Label"": ""Cass Corridor"", ""Type"": ""G"", ""Wiki..."
1255,1310,N35778,travel,travelnews,"China's largest airport hotel, Aerotel Hotel, ...","Aerotel Beijing has opened, and it's China's l...",https://assets.msn.com/labs/mind/AAIfSly.html,"[{""Label"": ""Beijing"", ""Type"": ""G"", ""WikidataId...","[{""Label"": ""Beijing"", ""Type"": ""G"", ""WikidataId..."
14048,14707,N62171,travel,travelnews,New Passenger Drop-Off Scheme Instituted At Ai...,There are new traffic rules for dropping off p...,https://assets.msn.com/labs/mind/AAJXXgI.html,"[{""Label"": ""St. Louis Lambert International Ai...","[{""Label"": ""St. Louis Lambert International Ai..."
37777,39694,N63004,travel,travelarticle,Orlando International Airport's fast-growing h...,Authorities at Orlando International Airport a...,https://assets.msn.com/labs/mind/BBWKrkx.html,"[{""Label"": ""Orlando International Airport"", ""T...","[{""Label"": ""Orlando International Airport"", ""T..."
29934,31402,N13589,foodanddrink,recipes,69 Vegetarian Meals to Make Meatless Monday Su...,"Chicken, who?",https://assets.msn.com/labs/mind/BBPq6e3.html,"[{""Label"": ""Vegetarianism"", ""Type"": ""C"", ""Wiki...",[]
...,...,...,...,...,...,...,...,...,...
37777,39694,N63004,travel,travelarticle,Orlando International Airport's fast-growing h...,Authorities at Orlando International Airport a...,https://assets.msn.com/labs/mind/BBWKrkx.html,"[{""Label"": ""Orlando International Airport"", ""T...","[{""Label"": ""Orlando International Airport"", ""T..."
2690,2809,N58055,news,newsopinion,We Are Running Out of Air,One suffocating city is a harbinger of health ...,https://assets.msn.com/labs/mind/AAJWQGS.html,[],[]
2998,3127,N723,tv,tv-gallery,"31 TV reboots, remakes, and spin-offs that are...","From classics like ""Grease"" to newer shows lik...",https://assets.msn.com/labs/mind/AAGsVHx.html,[],"[{""Label"": ""Gossip Girl"", ""Type"": ""W"", ""Wikida..."
4657,4865,N36954,foodanddrink,recipes,25+ Ways To Go Gluten-Free This Holiday,There's no bland health food on this list.,https://assets.msn.com/labs/mind/AAItUdd.html,"[{""Label"": ""Gluten-free diet"", ""Type"": ""C"", ""W...",[]


## Grouping

In [141]:
survey_groups = {
    'Group 1': ['A', 'C', 'E', 'G', 'I'],
    'Group 2': ['B', 'D', 'F', 'H', 'J'],
    'Group 3': ['K', 'L', 'O', 'A', 'B'],
    'Group 4': ['M', 'N', 'K', 'F', 'I']
}

In [142]:
all_clicks = survey_interactions_encoded.reset_index()

In [143]:
all_clicks

,index,user,item,rating
0,0,14904,760,1.0
1,1,1818,1195,1.0
2,2,1818,2576,1.0
3,3,7903,4833,1.0
4,4,14527,5728,1.0
...,...,...,...,...
155631,155631,14,3230,1.0
155632,155632,14,4978,1.0
155633,155633,14,5454,1.0
155634,155634,14,355,1.0


In [144]:
# Create an implicit feedback model with 50 features and 20 iterations
model = ImplicitMF(features=50, iterations=20, reg=0.1)

# Train the model
model.fit(all_clicks)

In [145]:
all_items = survey_interactions_encoded.item.unique().tolist()
survey_groups_recommendations = {}
explanations = []

In [146]:
for group_name, group in survey_groups.items():
    top_10_group_recs = group_recommendations(group, model, all_items, all_clicks)
    explanations.append(explain_group_recs(group, news, top_10_group_recs.index))
    survey_groups_recommendations[group_name] = top_10_group_recs

In [147]:
for group_name, recommendations in survey_groups_recommendations.items():
    print(group_name)
    print(recommendations)

Group 1
N59981    4.067606
N9720     3.986287
N36545    3.544328
N45509    3.498678
N60872    3.400545
N45270    3.339858
N30867    3.198268
N10470    3.174837
N13259    3.102246
N26025    3.078655
dtype: float64
Group 2
N59981    4.426622
N9720     4.315308
N45509    3.911443
N36545    3.846512
N60872    3.718481
N45270    3.645488
N13259    3.372795
N26025    3.329758
N3344     3.303623
N14780    3.228338
dtype: float64
Group 3
N59981    3.841956
N9720     3.748571
N45509    3.351378
N36545    3.337310
N60872    3.216385
N45270    3.157890
N62996    3.007842
N13259    2.924668
N26025    2.896792
N3344     2.860540
dtype: float64
Group 4
N59981    4.064218
N9720     3.956735
N45509    3.546237
N36545    3.524302
N60872    3.397343
N45270    3.335863
N11200    3.193473
N13259    3.088447
N26025    3.063222
N3344     3.019015
dtype: float64


In [148]:
for explanation in explanations:
    pretty_string = json.dumps(explanation, indent=0)
    print(f"{pretty_string}\n\n")

{
"N59981": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for tvnews.",
"N9720": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for lifestylecelebstyle.",
"N36545": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for tv-celebrity.",
"N45509": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for foodnews.",
"N60872": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for newsus.",
"N45270": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for newsworld.",
"N30867": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for causes-environment.",
"N10470": "Recommended because users ['A', 'C', 'E', 'G', 'I'] showed a high average preference score for tv-reviews.",
"N13259": "Recommended because 